# Identifying Missing Data in the Caltrans/PeMS Dataset

The California Department of Transportation (Caltrans) collects data that describes the flow of traffic on California freeways. Caltrans stores these data in a database called PeMS. The data describe the number of counts per unit time meaured by roughly 45,000 sensors on a 30-second cadence. The type of sensor varies considerably, e.g. radar and magnetometers (see Chapter 1 of the [Introduction to PeMS User Guide](https://pems.dot.ca.gov/Papers/PeMS_Intro_User_Guide_v6.pdf)). 

In some cases, these data are missing. Faulty or broken sensors do not collect data. Or sensor data is not wirelessly transmitted back to PeMS. In addition, Caltrans performs some calculations to convert these raw sensor data into physical observables such as speed. These calculations include some assumptions such as the length of the vehicle, or $g$. Based on the quality of the assumption, these data can include errors.

In this notebook, we will take a look at the nature of the missing data. Some questions to ask:
1. Are all the data available for the most recent year, 2023?
2. Are all the data available for the most recent decade, 2013-2023?
3. If data are missing, do they occur in any spatial or temporal clusters?

### Setup

In [1]:
import ibis
import os
import numpy as np
import pandas as pd
import seaborn as sns
import itertools

import matplotlib.pyplot as plt

from dotenv import load_dotenv
from functools import reduce
from datetime import datetime as dt_obj

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 3000)

In [2]:
load_dotenv(override=True)
USERNAME = os.getenv('USERNAME')
PASSWORD = os.getenv('PASSWORD')

In [3]:
con = ibis.snowflake.connect(
    user=USERNAME,
    password=PASSWORD,
    role='TRANSFORMER_DEV',
    warehouse='TRANSFORMING_XS_DEV',
    account="VSB79059-DSE_CALTRANS_PEMS",
    database="RAW_DEV/CLEARINGHOUSE",
)

In [4]:
station_metadata = con.table("STATION_META");
station_raw = con.table("STATION_RAW");
station_status = con.table("STATION_STATUS");

/Users/monica.bobra/Library/Caches/pypoetry/virtualenvs/caldata-mdsa-caltrans-pems-4CtDAEQU-py3.10/lib/python3.10/site-packages/ibis/backends/snowflake/__init__.py:268: UserWarning: Unable to create map UDFs, some functionality will not work: 003001 (42501): SQL access control error:
Insufficient privileges to operate on account 'NGB13288'
  warnings.warn(


In [5]:
station_metadata_df = station_metadata.execute(limit=10)
station_raw_df = station_raw.execute(limit=10)
station_status_df = station_status.execute(limit=10)

In [6]:
station_raw_df

,FILENAME,SAMPLE_TIMESTAMP,SAMPLE_DATE,ID,FLOW_1,OCCUPANCY_1,SPEED_1,FLOW_2,OCCUPANCY_2,SPEED_2,FLOW_3,OCCUPANCY_3,SPEED_3,FLOW_4,OCCUPANCY_4,SPEED_4,FLOW_5,OCCUPANCY_5,SPEED_5,FLOW_6,OCCUPANCY_6,SPEED_6,FLOW_7,OCCUPANCY_7,SPEED_7,FLOW_8,OCCUPANCY_8,SPEED_8
0,clhouse/raw/d04/2012/04/d04_text_station_raw_2...,2012-04-06 06:54:17,2012-04-06,401702,7,0.0361,71.0,4,0.0317,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,clhouse/raw/d04/2012/04/d04_text_station_raw_2...,2012-04-06 06:54:17,2012-04-06,401703,1,0.0067,65.0,3,0.0172,71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,clhouse/raw/d04/2012/04/d04_text_station_raw_2...,2012-04-06 06:54:17,2012-04-06,401704,0,0.0000,0.0,0,0.0000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,clhouse/raw/d04/2012/04/d04_text_station_raw_2...,2012-04-06 06:54:17,2012-04-06,401706,0,0.0000,0.0,0,0.0000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,clhouse/raw/d04/2012/04/d04_text_station_raw_2...,2012-04-06 06:54:17,2012-04-06,401710,14,0.0917,78.0,9,0.0583,71.0,6.0,0.0794,65.0,5.0,0.0639,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,clhouse/raw/d04/2012/04/d04_text_station_raw_2...,2012-04-06 06:54:17,2012-04-06,401716,9,0.0528,71.0,20,0.1511,60.0,17.0,0.1961,47.0,16.0,0.1950,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,clhouse/raw/d04/2012/04/d04_text_station_raw_2...,2012-04-06 06:54:17,2012-04-06,401734,0,1.0000,0.0,0,1.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
7,clhouse/raw/d04/2012/04/d04_text_station_raw_2...,2012-04-06 06:54:17,2012-04-06,401742,0,0.0000,0.0,0,0.0000,0.0,0.0,1.0000,0.0,0.0,0.0000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,clhouse/raw/d04/2012/04/d04_text_station_raw_2...,2012-04-06 06:54:17,2012-04-06,401748,0,0.0000,0.0,0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,clhouse/raw/d04/2012/04/d04_text_station_raw_2...,2012-04-06 06:54:17,2012-04-06,401762,0,0.0000,0.0,0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


### Question 1. Are all the data available for the most recent year, 2023?

Case Study 1: Look at District 5 from the [District Map and County Chart](https://cwwp2.dot.ca.gov/documentation/district-map-county-chart.htm) data during August 2023.

1. Filter the station metadata file to look at `DISTRICT` values of 5.
2. Strip away all old versions of the data by selecting the most recent unique ID.
3. Filter raw data file to look at August 2023 and the selected IDs from Step 2.
4. Merge the raw and metadata files on the keyword `ID`.
5. Look at the keyword `SAMPLE_TIMESTAMP` in the `STATION_RAW` table.
6. Plot `SAMPLE_TIMESTAMP` for one highway for one day.

##### 1. Filter the station metadata file to look at `DISTRICT` values of 5.

In [7]:
district_5_filter = station_metadata.filter(station_metadata["DISTRICT"] == "5")

In [8]:
district_5_df = district_5_filter.execute()

##### 2. Strip away all old versions of the data by selecting the most recent unique ID.

Identify the number of unique values of ID in District 5.

In [9]:
print('There are {} unique values of ID in District 5.'.format(district_5_df['ID'].nunique()))

There are 706 unique values of ID in District 5.


Construct a new column called `DATA_VERSION`.

In [22]:
district_5_df['DATA_VERSION'] = np.NaN

Extract the date from the `FILENAME` keyword. Populate these dates in the `DATA_VERSION` keyword.

In [34]:
for i in range(len(district_5_df)):
    data_version = dt_obj.strptime(district_5_df['FILENAME'][0][39:49], '%Y_%m_%d')
    district_5_df['DATA_VERSION'][i] = data_version

Select the most recent `DATA_VERSION` for each unique ID. Drop the rest.

In [24]:
unique_IDs = district_5_df['ID'].value_counts().index.to_list()

In [25]:
drop_these_rows = []
for i in range(len(unique_IDs)):
    ID_subset = district_5_df[district_5_df['ID'] == unique_IDs[i]]
    index_for_max_value = ID_subset['DATA_VERSION'].idxmax()
    indices_for_rows_to_drop = ID_subset.drop(index_for_max_value).index.to_list()
    drop_these_rows.append(indices_for_rows_to_drop)

In [26]:
drop_these_rows_flattened = list(itertools.chain.from_iterable(drop_these_rows))

In [27]:
district_5_recent_version_df = district_5_df.drop(drop_these_rows_flattened).reset_index(drop=True)

##### 3. Merge the raw and metadata files on the keyword `ID`.

In [28]:
# Randomly select an ID
ID_selection = district_5_recent_version_df['ID'][2]
print(ID_selection)

500004


In [ ]:
# Get the date range for this ID
date_and_ID_first_sample_date = station_raw.select('SAMPLE_DATE', 'ID').filter(station_raw["ID"] == ID_selection).order_by('SAMPLE_DATE')
date_and_ID_last_sample_date = station_raw.filter(station_raw["ID"] == ID_selection).select('SAMPLE_DATE', 'ID').order_by(ibis.desc('SAMPLE_DATE'))

In [ ]:
date_and_ID_last_sample_date

In [ ]:
first_sample_date = date_and_ID_first_sample_date.execute(limit=1)
last_sample_date = date_and_ID_last_sample_date.execute(limit=1)

In [ ]:
print(first_sample_date)

In [ ]:
print(last_sample_date)

This ID only looks at data from 2012, so we can drop it. Go through all the IDs and select the ones with data in 2023.

In [29]:
date_selection_start = dt_obj(2023, 1, 1)
date_selection_end = dt_obj(2023, 12, 31)

In [30]:
ID_selection = district_5_recent_version_df['ID'][0]

In [31]:
date_and_ID_filter = station_raw.filter(
    (station_raw["SAMPLE_DATE"] >= date_selection_start) & (station_raw["SAMPLE_DATE"] < date_selection_end)).filter(
    station_raw["ID"] == ID_selection)

In [32]:
date_and_ID_filter

r0 := DatabaseTable: STATION_RAW
  FILENAME         string
  SAMPLE_TIMESTAMP timestamp
  SAMPLE_DATE      date
  ID               string
  FLOW_1           int64
  OCCUPANCY_1      float64
  SPEED_1          float64
  FLOW_2           int64
  OCCUPANCY_2      float64
  SPEED_2          float64
  FLOW_3           int64
  OCCUPANCY_3      float64
  SPEED_3          float64
  FLOW_4           int64
  OCCUPANCY_4      float64
  SPEED_4          float64
  FLOW_5           int64
  OCCUPANCY_5      float64
  SPEED_5          float64
  FLOW_6           int64
  OCCUPANCY_6      float64
  SPEED_6          float64
  FLOW_7           int64
  OCCUPANCY_7      float64
  SPEED_7          float64
  FLOW_8           int64
  OCCUPANCY_8      float64
  SPEED_8          float64

Selection[r0]
  predicates:
    r0.SAMPLE_DATE >= datetime.datetime(2023, 1, 1, 0, 0)
    r0.SAMPLE_DATE < datetime.datetime(2023, 12, 31, 0, 0)
    r0.ID == '500001'